## Setup

1. Complete the previous challenge.
2. Create a Cognitive Services resource.

In [5]:
# Azure Storage account settings:
azure_storage_account = 'YOUR_STORAGE_ACCOUNT'
blob_connection_string = "YOUR_BLOB_STORE_CONNECTION_STRING"
blob_container = "YOUR_BLOB_CONTAINER"

# Azure Search constants
azsearch_url = "YOUR_SEARCH_SERVICE.search.windows.net"
azsearch_key = "YOUR_SEARCH_KEY"

# Cognitive Services Key
cog_svc_key = "YOUR_COGNITIVE_SERVICE_KEY"

## Function to wrap HTTP requests

In [2]:
def azsearch_rest(request_type="GET", function_name="servicestats", body=None):
    # Imports and constants
    import http.client, urllib.request, urllib.parse, urllib.error, base64, json, urllib

    # Request headers.
    headers = {
        'Content-Type': 'application/json',
        'api-key': azsearch_key
    }

    # Request parameters
    params = urllib.parse.urlencode({
        'api-version':'2017-11-11-Preview'
    })
    
    try:
        # Execute the REST API call and get the response.
        conn = http.client.HTTPSConnection(azsearch_url)
        request_path = "/{0}?{1}".format(function_name, params)
        conn.request(request_type, request_path, body, headers)
        response = conn.getresponse()
        print(response.status)
        data = response.read().decode("UTF-8")
        result = None
        if len(data) > 0:
            result = json.loads(data)
        return result

    except Exception as ex:
        raise ex

print("Ready to use the REST API")

Ready to use the REST API


## Verify Data Source Exists

In [3]:
import json

try:
    result = azsearch_rest(function_name="datasources")
    if len(result) > 0:
        print(json.dumps(result, sort_keys=True, indent=2))

except Exception as e:
    print('Error:')
    print(e)

200
{
  "@odata.context": "https://azsrch-margies.search.windows.net/$metadata#datasources",
  "value": [
    {
      "@odata.etag": "\"0x8D6C45C6B278563\"",
      "container": {
        "name": "margies-data",
        "query": null
      },
      "credentials": {
        "connectionString": null
      },
      "dataChangeDetectionPolicy": null,
      "dataDeletionDetectionPolicy": null,
      "description": "Various documents.",
      "name": "website-docs",
      "subtype": null,
      "type": "azureblob"
    }
  ]
}


## Update the Index

In [4]:
import json

# Define the request body
body = {
  "fields": [
    {
      # Unique ID for each document
      "name": "id",
      "type": "Edm.String",
      "key": True,
      "searchable": True,
      "filterable": True,
      "facetable": False,
      "sortable": True
    },
    {
     # The file path of the document
      "name": "url",
      "type": "Edm.String",
      "sortable": True,
      "searchable": True,
      "filterable": True,
      "facetable": False
    },
    {
      # The file name of the document
      "name": "file_name",
      "type": "Edm.String",
      "sortable": True,
      "searchable": True,
      "filterable": True,
      "facetable": False
    },
    {
      # The size of the document
      "name": "size",
      "type": "Edm.Int64",
      "sortable": True,
      "searchable": False,
      "filterable": True,
      "facetable": False
    },
    {
      # The last-modified date of the document
      "name": "last_modified",
      "type": "Edm.DateTimeOffset",
      "sortable": True,
      "searchable": False,
      "filterable": True,
      "facetable": False
    },
    {
      # The text content in the document
      "name": "content",
      "type": "Edm.String",
      "sortable": False,
      "searchable": True,
      "filterable": True,
      "facetable": False
    },
    {
      # The sentiment score
      "name": "sentiment",
      "type": "Edm.Double",
      "searchable": False,
      "sortable": True,
      "filterable": True,
      "facetable": True
    },
    {
      # Key phrases that might help identify the subject of the text
      "name": "key_phrases",
      "type": "Collection(Edm.String)",
      "searchable": True,
      "filterable": False,
      "facetable": False
    },
    {
      # Geographical locations mentioned in the text
      "name": "locations",
      "type": "Collection(Edm.String)",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False
    },
    {
      # People mentioned in the text
      "name": "people",
      "type": "Collection(Edm.String)",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False
    },
    {
      # URLs mentioned in the text
      "name": "links",
      "type": "Collection(Edm.String)",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False
    },
    {
      # URLs mentioned in the text
      "name": "entities",
      "type": "Collection(Edm.String)",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False
    }
  ]
}  

try:
    result = azsearch_rest(request_type="PUT", function_name="indexes/website-docs-index", body=json.dumps(body))
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))

except Exception as e:
    print('Error:')
    print(e)
    

204


## Define a Skillset

In [5]:
# Define the request body
body = {
  "description": "Extract and enhance text",
  "skills":
  [
         # Detect the sentiment
        {
          "description": "Detect sentiment.",
          "@odata.type": "#Microsoft.Skills.Text.SentimentSkill",
          "defaultLanguageCode": "en",
          "inputs": [
            {
              "name": "text",
              "source": "/document/content"
            }
          ],
            "outputs": [
                {
                  "name": "score",
                  "targetName": "sentimentScore"
                }
            ]
        },
       # Extract entities
        {
          "@odata.type": "#Microsoft.Skills.Text.EntityRecognitionSkill",
          "categories": [ "Organization", "location", "person", "datetime", "url" ],
          "defaultLanguageCode": "en",
          "inputs": [
            {
              "name": "text",
              "source": "/document/content"
            }
          ],
          "outputs": [
            {
              "name": "locations"
            },
            {
              "name": "persons"
            },
            {
              "name": "urls"
            },
            {
              "name": "entities"
            }
          ]
        },
      # Extract key phrases from the pages
      {
          "@odata.type": "#Microsoft.Skills.Text.KeyPhraseExtractionSkill",
          "context": "/document",
          "defaultLanguageCode": "en",
          "maxKeyPhraseCount": 5,
          "inputs": [
            {
              "name": "text", "source": "/document/content"
            }
          ],
          "outputs": [
            {
              "name": "keyPhrases"
            }
          ]
        },  
      ],
      "cognitiveServices": {
        "@odata.type": "#Microsoft.Azure.Search.CognitiveServicesByKey",
        "description": "margies-cog-svc",
        "key": cog_svc_key
      }
    }  

try:
    result = azsearch_rest(request_type="PUT", function_name="/skillsets/website-skillset", body=json.dumps(body))
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))

except Exception as e:
    print('Error:')
    print(e)

204


## Create an Indexer

In [6]:
import json

body = {
  "name":"website-docs-indexer", 
  "dataSourceName" : "website-docs",
  "skillsetName" : "website-skillset",
  "targetIndexName" : "website-docs-index",
  "fieldMappings" : [
      # Map the soure content fields to index fields
        {
          "sourceFieldName" : "metadata_storage_path",
          "targetFieldName" : "id",
          "mappingFunction" : 
            { "name" : "base64Encode" }
        },
        {
          "sourceFieldName" : "metadata_storage_path",
          "targetFieldName" : "url"
        },
        {
          "sourceFieldName" : "metadata_storage_name",
          "targetFieldName" : "file_name"
        },
        {
          "sourceFieldName" : "content",
          "targetFieldName" : "content"
        },
        {
          "sourceFieldName" : "metadata_storage_size",
          "targetFieldName" : "size"
        },
        {
          "sourceFieldName" : "metadata_storage_last_modified",
          "targetFieldName" : "last_modified"
        }
   ],
  "outputFieldMappings" : 
  [
      # Map the output from cognitive skills to index fields
        {
            "sourceFieldName": "/document/sentimentScore", 
            "targetFieldName": "sentiment"
        },
        {
          "sourceFieldName" : "/document/persons", 
          "targetFieldName" : "people"
        },
        {
          "sourceFieldName" : "/document/locations", 
          "targetFieldName" : "locations"
        },
        {
          "sourceFieldName" : "/document/urls", 
          "targetFieldName" : "links"
        },
        {
          "sourceFieldName" : "/document/entities", 
          "targetFieldName" : "entities"
        },
        {
          "sourceFieldName" : "/document/keyPhrases/*", 
          "targetFieldName" : "key_phrases"
        }
  ],
  "parameters":
  {
    "maxFailedItems":-1,
    "maxFailedItemsPerBatch":-1,
    "configuration": 
    {
        "dataToExtract": "contentAndMetadata"
    }
  }
}  

try:
    result = azsearch_rest(request_type="PUT", function_name="indexers/website-docs-indexer", body=json.dumps(body))
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))

except Exception as e:
    print('Error:')
    print(e)

204


In [7]:
import time

try:
    # Reset the indexer.
    result = azsearch_rest(request_type="POST", function_name="/indexers/website-docs-indexer/reset", body=None)
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))
        
    # re-run the indexer.
    result = azsearch_rest(request_type="POST", function_name="/indexers/website-docs-indexer/run", body=None)
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))
    


except Exception as e:
    print('Error:')
    print(e)

204
202


In [8]:
import time, json

try:
    complete = False
    while (complete == False):
        result = azsearch_rest(request_type="GET", function_name="indexers/website-docs-indexer/status")
        state = result["status"]
        if result['lastResult'] is not None:
            state = result['lastResult']['status']
        print (state)
        if state in ("success", "error"):
            complete = True
        time.sleep(1)

except Exception as e:
    print('Error:')
    print(e)

200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
success


## Create Wrapper Function for Query Requests

In [6]:
def azsearch_query(index, params):
    # Imports and constants
    import http.client, urllib.request, urllib.parse, urllib.error, base64, json, urllib

    # Request headers.
    headers = {
        'Content-Type': 'application/json',
        'api-key': azsearch_key
    }
    
    try:
        # Execute the REST API call and get the response.
        conn = http.client.HTTPSConnection(azsearch_url)
        request_path = "/indexes/{0}/docs?{1}".format(index, params)
        conn.request("GET", request_path, None, headers)
        response = conn.getresponse()
        data = response.read().decode("UTF-8")
        result = json.loads(data)
        return result

    except Exception as ex:
        raise ex

print("Ready to use the REST API for Queries")

Ready to use the REST API for Queries


## Queries

### Reviews Sentiment and Key Phrases

In [14]:
import urllib.parse, json

search_terms = input("Search: ")

# Define the search parameters
searchParams = urllib.parse.urlencode({
    'search':'"{0}"'.format(search_terms),
    'searchMode':'All',
    '$count':'true',
    'queryType':'full',
    '$select':'url, file_name, key_phrases, sentiment',
    '$filter': "search.ismatch('reviews', 'url')",
    '$orderby': 'sentiment desc',
    'api-version':'2017-11-11-Preview'
})

try:
    result = azsearch_query(index="website-docs-index", params=searchParams)
    print('Hits:',result['@odata.count'])
    for value in result["value"]:
        print("\n", value["url"])
        print(value["file_name"], " - Sentiment:", value["sentiment"])
        print(value["key_phrases"])

except Exception as e:
    print('Error:')
    print(e)

Search: New York
Hits: 16

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201828.pdf
201828.pdf  - Sentiment: 0.9961904287338256
['Friendly Hotel', 'Grand Central Hotel', 'perfect hotel', 'New York standards', 'Grand Central Station']

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201860.pdf
201860.pdf  - Sentiment: 0.9920151233673096
['room coffee', 'nice room', 'room safe', 'Park Hotel', 'busy hotel']

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201838.pdf
201838.pdf  - Sentiment: 0.9881517887115479
['lovely hotel', 'Grand Central Hotel', 'Great New York Boutique Hotel', 'lovely surprise', 'lovely touch']

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201815.pdf
201815.pdf  - Sentiment: 0.9881031513214112
['Manhattan Hotel', 'Great Hotel', 'typical New York City hotel', 'energy of New York city', 'best possible location']

 https://azstoremargies.bl

### City and Locations

In [10]:
import urllib.parse, json

city = input("City: ")
locations = input("Locations: ")

# Define the search parameters
searchParams = urllib.parse.urlencode({
    # Search for "London"
    'search':'"{0}" + "{1}"'.format(city, locations),
    'searchMode':'All',
    '$count':'true',
    'queryType':'full',
    '$select':'url, file_name, sentiment, locations',
    '$filter': "search.ismatch('reviews', 'url')",
    'api-version':'2017-11-11-Preview'
})

try:
    result = azsearch_query(index="website-docs-index", params=searchParams)
    print('Hits:',result['@odata.count'])
    for value in result["value"]:
        print("\n", value["url"])
        print(value["file_name"], " - Sentiment:", value["sentiment"])
        print(value["locations"])


except Exception as e:
    print('Error:')
    print(e)

City: New York
Locations: Broadway
Hits: 2

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201826.pdf
201826.pdf  - Sentiment: 0.8254959583282471
['Manhattan', 'Manhattan Hotel', 'New York', 'New York, USA', 'Central Park', 'Broadway']

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201858.pdf
201858.pdf  - Sentiment: 0.19390404224395752
['Park Hotel', 'USA', 'US', 'LA', 'New York', 'Melbourne', 'Broadway', 'Heaven']


### Reviews by Reviewer

In [17]:
import urllib.parse, json

reviewer = input("Reviewer: ")

# Define the search parameters
searchParams = urllib.parse.urlencode({
    'search':'"{0}"'.format(reviewer),
    'searchMode':'All',
    '$count':'true',
    'queryType':'full',
    '$select':'file_name, url, locations, key_phrases, sentiment',
    '$filter': "search.ismatch('reviews', 'url')",
    'api-version':'2017-11-11-Preview'
})

try:
    result = azsearch_query(index="website-docs-index", params=searchParams)
    print('Hits:',result['@odata.count'])
    for value in result["value"]:
        print("\n", value["url"])
        print(value["file_name"], " - Sentiment:", value["sentiment"])
        print(value["locations"])
        print(value["key_phrases"])


except Exception as e:
    print('Error:')
    print(e)

Reviewer: Matthew Daughtry
Hits: 2

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201818.pdf
201818.pdf  - Sentiment: 0.02185213565826416
['Kensington', 'London', 'London, UK', 'linen', 'mould']
['family room', 'room cold', 'bed linen dirty', 'smallest double room', 'worst hotel']

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201812.pdf
201812.pdf  - Sentiment: 0.9672240018844604
['San Francisco', 'San Francisco, USA', 'SF', 'Lombard', 'us', 'Golden gate bridge', 'MUNI', 'American', 'pas']
['Good base', 'MUNI buses rest', 'good value', 'helpful staff', 'San Francisco']


### Negative Reviews

In [18]:
import urllib.parse, json

search_terms = input("Search: ")

# Define the search parameters
searchParams = urllib.parse.urlencode({
    # Search for "Las Vegas" where URL doesn't include "reviews"
    'search':'"{0}"'.format(search_terms),
    'searchMode':'All',
    '$count':'true',
    'queryType':'full',
    '$select':'url, file_name, key_phrases, sentiment',
    '$filter': "search.ismatch('reviews', 'url') and sentiment le 0.5",
    '$orderby': 'sentiment desc',
    'api-version':'2017-11-11-Preview'
})

try:
    result = azsearch_query(index="website-docs-index", params=searchParams)
    print('Hits:',result['@odata.count'])
    for value in result["value"]:
        print("\n", value["url"])
        print(value["file_name"], " - Sentiment:", value["sentiment"])
        print(value["key_phrases"])

except Exception as e:
    print('Error:')
    print(e)

Search: Las Vegas
Hits: 5

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201853.pdf
201853.pdf  - Sentiment: 0.5
['strip', 'Fountain Hotel', 'bedrooms', 'Rooms clean', 'Las Vegas']

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201856.pdf
201856.pdf  - Sentiment: 0.5
['Las Vegas', 'Fountain Hotel', 'level', 'activities', 'USA']

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201848.pdf
201848.pdf  - Sentiment: 0.23280319571495056
['Las Vegas', 'check', 'Fountain Hotel', 'USA', 'Reviewer']

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201862.pdf
201862.pdf  - Sentiment: 0.23071569204330444
['Dubai', 'star hotel', 'vegas hotel', 'Lost City Hotel', 'water swim']

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201843.pdf
201843.pdf  - Sentiment: 0.17904305458068848
['room', 'day', 'Fountain Hotel', 'expensive hotel', 'Hotel

### Collateral Containing Links

In [21]:
import urllib.parse, json

search_terms = input("Search: ")

# Define the search parameters
searchParams = urllib.parse.urlencode({
    'search':'"{0}"'.format(search_terms),
    'searchMode':'All',
    '$count':'true',
    '$filter': "search.ismatch('collateral', 'url') and links/any()",
    'queryType':'full',
    '$select':'url, file_name, links',
    'api-version':'2017-11-11-Preview'
})

try:
    result = azsearch_query(index="website-docs-index", params=searchParams)
    print('Hits:',result['@odata.count'])
    for value in result["value"]:
        print("\n", value["url"])
        print(value["file_name"])
        print(value["links"])

except Exception as e:
    print('Error:')
    print(e)

Search: San Francisco
Hits: 2

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/collateral/San%20Francisco%20Brochure.pdf
San Francisco Brochure.pdf
['www.margiestravel.com']

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/collateral/Margies%20Travel%20Company%20Info.pdf
Margies Travel Company Info.pdf
['www.margiestravel.com']
